In [1]:
#Tensorflow version

import tensorflow as tf
print(tf.__version__)

2.9.1


In [2]:
#Importing libraries

import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Conv2D, Dense, Flatten, BatchNormalization, Dropout, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model
import matplotlib.pyplot as plt
from keras.applications.efficientnet import EfficientNetB0
from keras.preprocessing import image
#from keras.preprocessing.image import load_img
from keras.models import Sequential
import numpy as np
from tensorflow.keras import regularizers

In [3]:
#Datasets path

#17000 samples original dataset

train_dir='/home/ubuntu/From_Laptop/multi-class-NEW/12-Lead_ECG/12-Leads-Graphs/Training/'
val_dir='/home/ubuntu/From_Laptop/multi-class-NEW/12-Lead_ECG/12-Leads-Graphs/Validation/'


In [4]:
from tensorflow.keras import layers

train_data = tf.keras.preprocessing.image_dataset_from_directory(directory=train_dir,
                                                                 image_size=(256,256),
                                                                 label_mode='categorical', batch_size=128, 
                                                                 )
val_data = tf.keras.preprocessing.image_dataset_from_directory(directory=val_dir,
                                                                 image_size=(256,256),
                                                                 label_mode='categorical', batch_size=128,
                                                                )

Found 13040 files belonging to 5 classes.


2023-02-17 09:22:34.895969: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-17 09:22:34.986957: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-17 09:22:34.987273: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-17 09:22:35.018319: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Found 5179 files belonging to 5 classes.


In [5]:
#Normalization of the data

nrmzln_layer = layers.experimental.preprocessing.Rescaling(1./255)

train_data = train_data.map(lambda x,
                          y: (nrmzln_layer(x), y))

val_data = val_data.map(lambda x,
                          y: (nrmzln_layer(x), y))

In [6]:
# Instantiate base_model with pre-trained weights

#import tensorflow.python.keras.applications.VGG16

base_model= tf.keras.applications.VGG16(weights= 'imagenet', input_shape=(256,256, 3), include_top=False) 

In [7]:
# Freezing the base_model for transfer learning
base_model.trainable= False

In [8]:
# Creating a new_model on the top
inputs= tf.keras.Input(shape= (256, 256, 3), name="input_layer")

In [9]:
x= base_model(inputs,training= False)

In [10]:
# Average pool the outputs of the base model
x = tf.keras.layers.GlobalAveragePooling2D(name='Global_Average_Pooling_Layer')(x)

In [11]:
# Output layer
u = tf.keras.layers.Dense(1050, activation="relu")(x)
u = tf.keras.layers.Dense(512, activation="relu")(u)
u = tf.keras.layers.Dense(256, activation="relu")(u)
u = tf.keras.layers.Dense(128, activation="relu")(u)
outputs = tf.keras.layers.Dense(5, activation="softmax",name="output_layer")(u)

In [12]:
# Combine the model
model = tf.keras.Model(inputs, outputs) 

In [13]:
#printing model summary

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 256, 256, 3)]     0         
                                                                 
 vgg16 (Functional)          (None, 8, 8, 512)         14714688  
                                                                 
 Global_Average_Pooling_Laye  (None, 512)              0         
 r (GlobalAveragePooling2D)                                      
                                                                 
 dense (Dense)               (None, 1050)              538650    
                                                                 
 dense_1 (Dense)             (None, 512)               538112    
                                                                 
 dense_2 (Dense)             (None, 256)               131328    
                                                             

In [14]:
# Compile the model

from tensorflow.keras.metrics import SensitivityAtSpecificity,SpecificityAtSensitivity,Recall,Precision

model.compile(loss= tf.losses.categorical_crossentropy, 
      optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
              metrics=['accuracy',SensitivityAtSpecificity(0.5),SpecificityAtSensitivity(0.5),Recall(0.5),Precision(0.5)])

In [15]:
#callback

class myCallback(tf.keras.callbacks.Callback):
     def on_epoch_end(self, epoch, logs={}):
        if(logs.get('val_accuracy')>0.99): 
          print("\nReached 99% accuracy so cancelling training!")
          self.model.stop_training = True

callbacks = myCallback()

In [16]:
# Fit the model
from keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = ModelCheckpoint("ECG_Spectrogram_Model.h5", monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
history = model.fit(train_data, steps_per_epoch=len(train_data), epochs=50, validation_data=val_data, validation_steps=len(val_data),callbacks = [callbacks,checkpoint])

Epoch 1/50


2023-02-17 09:22:50.640491: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2023-02-17 09:22:51.185732: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


102/102 [==============================] - ETA: 0s - loss: 1.2747 - accuracy: 0.4777 - sensitivity_at_specificity: 0.8567 - specificity_at_sensitivity: 0.8636 - recall: 0.2151 - precision: 0.6408
Epoch 1: val_accuracy improved from -inf to 0.53524, saving model to ECG_Spectrogram_Model.h5
102/102 [==============================] - 105s 914ms/step - loss: 1.2747 - accuracy: 0.4777 - sensitivity_at_specificity: 0.8567 - specificity_at_sensitivity: 0.8636 - recall: 0.2151 - precision: 0.6408 - val_loss: 1.1584 - val_accuracy: 0.5352 - val_sensitivity_at_specificity: 0.9004 - val_specificity_at_sensitivity: 0.9004 - val_recall: 0.3698 - val_precision: 0.6467
Epoch 2/50
102/102 [==============================] - ETA: 0s - loss: 1.0747 - accuracy: 0.5699 - sensitivity_at_specificity: 0.9264 - specificity_at_sensitivity: 0.9266 - recall: 0.4102 - precision: 0.6949
Epoch 2: val_accuracy improved from 0.53524 to 0.58293, saving model to ECG_Spectrogram_Model.h5
102/102 [========================

Epoch 14/50
102/102 [==============================] - ETA: 0s - loss: 0.8897 - accuracy: 0.6568 - sensitivity_at_specificity: 0.9663 - specificity_at_sensitivity: 0.9617 - recall: 0.5548 - precision: 0.7373
Epoch 14: val_accuracy improved from 0.65090 to 0.65302, saving model to ECG_Spectrogram_Model.h5
102/102 [==============================] - 83s 813ms/step - loss: 0.8897 - accuracy: 0.6568 - sensitivity_at_specificity: 0.9663 - specificity_at_sensitivity: 0.9617 - recall: 0.5548 - precision: 0.7373 - val_loss: 0.9136 - val_accuracy: 0.6530 - val_sensitivity_at_specificity: 0.9564 - val_specificity_at_sensitivity: 0.9629 - val_recall: 0.5495 - val_precision: 0.7350
Epoch 15/50
102/102 [==============================] - ETA: 0s - loss: 0.8903 - accuracy: 0.6566 - sensitivity_at_specificity: 0.9628 - specificity_at_sensitivity: 0.9622 - recall: 0.5525 - precision: 0.7377
Epoch 15: val_accuracy improved from 0.65302 to 0.66017, saving model to ECG_Spectrogram_Model.h5
102/102 [=======

Epoch 27/50
102/102 [==============================] - ETA: 0s - loss: 0.8329 - accuracy: 0.6838 - sensitivity_at_specificity: 0.9722 - specificity_at_sensitivity: 0.9698 - recall: 0.5932 - precision: 0.7531
Epoch 27: val_accuracy did not improve from 0.67233
102/102 [==============================] - 82s 807ms/step - loss: 0.8329 - accuracy: 0.6838 - sensitivity_at_specificity: 0.9722 - specificity_at_sensitivity: 0.9698 - recall: 0.5932 - precision: 0.7531 - val_loss: 0.9032 - val_accuracy: 0.6646 - val_sensitivity_at_specificity: 0.9577 - val_specificity_at_sensitivity: 0.9661 - val_recall: 0.5943 - val_precision: 0.7344
Epoch 28/50
102/102 [==============================] - ETA: 0s - loss: 0.8309 - accuracy: 0.6803 - sensitivity_at_specificity: 0.9729 - specificity_at_sensitivity: 0.9704 - recall: 0.5888 - precision: 0.7531
Epoch 28: val_accuracy did not improve from 0.67233
102/102 [==============================] - 82s 808ms/step - loss: 0.8309 - accuracy: 0.6803 - sensitivity_at

Epoch 40/50
102/102 [==============================] - ETA: 0s - loss: 0.7733 - accuracy: 0.7067 - sensitivity_at_specificity: 0.9801 - specificity_at_sensitivity: 0.9770 - recall: 0.6260 - precision: 0.7713
Epoch 40: val_accuracy did not improve from 0.67851
102/102 [==============================] - 82s 807ms/step - loss: 0.7733 - accuracy: 0.7067 - sensitivity_at_specificity: 0.9801 - specificity_at_sensitivity: 0.9770 - recall: 0.6260 - precision: 0.7713 - val_loss: 0.8904 - val_accuracy: 0.6648 - val_sensitivity_at_specificity: 0.9683 - val_specificity_at_sensitivity: 0.9643 - val_recall: 0.6032 - val_precision: 0.7167
Epoch 41/50
102/102 [==============================] - ETA: 0s - loss: 0.7625 - accuracy: 0.7080 - sensitivity_at_specificity: 0.9825 - specificity_at_sensitivity: 0.9768 - recall: 0.6324 - precision: 0.7717
Epoch 41: val_accuracy did not improve from 0.67851
102/102 [==============================] - 82s 807ms/step - loss: 0.7625 - accuracy: 0.7080 - sensitivity_at

In [17]:
#Acquiring Performance metrics

Training_Accuracy=history.history['accuracy']
Validation_Accuracy=history.history['val_accuracy']
Validation_Specificity=history.history['val_specificity_at_sensitivity']
Validation_Sensitivity=history.history['val_sensitivity_at_specificity']
Validation_Recall=history.history['val_recall']
Validation_Precision=history.history['val_precision']
Validation_Loss=history.history['val_loss']

print("Training Accuracy: ",max(Training_Accuracy))
print("Validation Accuracy: ",max(Validation_Accuracy))
print("Validation Specificity: ",max(Validation_Specificity))
print("Validation Sensitivity: ",max(Validation_Sensitivity))
print("Validation Recall: ",max(Validation_Recall))
print("Validation Precision: ",max(Validation_Precision))
print("Validation Loss: ",min(Validation_Loss))

Training Accuracy:  0.7305215001106262
Validation Accuracy:  0.684688150882721
Validation Specificity:  0.9713265299797058
Validation Sensitivity:  0.9683336615562439
Validation Recall:  0.6327476501464844
Validation Precision:  0.7468982338905334
Validation Loss:  0.8785406947135925
